# Flood WebMap App

## Create images from ASC files

In [2]:
import os
import re
import numpy as np
import rasterio
from rasterio.transform import from_origin
from rasterio.io import MemoryFile
from rasterio.warp import calculate_default_transform, reproject, Resampling
from pyproj import Transformer
from PIL import Image, ImageDraw, ImageFont

# ====================================================
# PARAMETERS (modify these values as needed)
# ====================================================

INPUT_DIR = "/Users/cmgotelli/Desktop/dems"      # Directory containing IBER ASC files
OUTPUT_DIR = "/Users/cmgotelli/Desktop/dems"     # Directory where PNGs will be saved
USER_MAX_DEPTH = 2.0                             # Maximum depth value (in your data units)
SRC_CRS = "EPSG:2056"                            # Source coordinate reference system
DST_CRS = "EPSG:3857"                            # Destination CRS for the PNG output

# Legend parameters:
LEGEND_WIDTH = 50      # Total width (in pixels) of the legend image
LEGEND_HEIGHT = 256    # Total height (in pixels) of the legend image
TOP_MARGIN = 40        # Space at the top for the title
BOTTOM_MARGIN = 20     # Space at the bottom for labels
LEFT_MARGIN = 25       # Left margin (for the color bar area)
RIGHT_MARGIN = 10      # Right margin (for the color bar area)
# The color bar will occupy the vertical space between TOP_MARGIN and (LEGEND_HEIGHT - BOTTOM_MARGIN)
# Values are scaled from 0 to 0.9*USER_MAX_DEPTH.
# Note: The legend image background will be white with 50% transparency.

# ====================================================
# FUNCTION DEFINITIONS
# ====================================================

def read_ascii_xllcenter(filename):
    """
    Reads an Arc ASCII file with a header using XLLCENTER/YLLCENTER.
    Returns:
      - data: 2D numpy array of the raster values
      - transform: Affine transform from rasterio
      - ncols: number of columns
      - nrows: number of rows
      - nodata_val: NoData value (float)
    Assumes that the first data line corresponds to the top row.
    """
    with open(filename, 'r') as f:
        header_lines = [next(f) for _ in range(6)]
    
    ncols = nrows = None
    xllcenter = yllcenter = None
    cellsize = None
    nodata_val = None

    for line in header_lines:
        line = line.strip()
        parts = re.split(r"\s+", line)
        key = parts[0].upper()
        val = float(parts[-1])
        if key.startswith("NCOLS"):
            ncols = int(val)
        elif key.startswith("NROWS"):
            nrows = int(val)
        elif key.startswith("XLLCENTER"):
            xllcenter = val
        elif key.startswith("YLLCENTER"):
            yllcenter = val
        elif key.startswith("CELLSIZE"):
            cellsize = val
        elif key.startswith("NODATA_VALUE"):
            nodata_val = val

    data = np.loadtxt(filename, skiprows=6)
    # Compute the upper-left corner (x_min, y_max)
    x_min = xllcenter - 0.5 * cellsize
    y_max = yllcenter + (nrows - 0.5) * cellsize
    transform = from_origin(x_min, y_max, cellsize, cellsize)
    return data, transform, ncols, nrows, nodata_val

def asc_to_png(filename_asc, output_png, src_crs=SRC_CRS, dst_crs=DST_CRS, global_max=USER_MAX_DEPTH):
    """
    Converts an ASC file to a PNG applying a colormap that interpolates from light blue to dark blue
    for values between 0 and 90% of global_max. Values above global_max are rendered in black.
    Returns the bounding box in EPSG:4326.
    """
    # 1) Read the ASC file.
    data, transform, width, height, nodata_val = read_ascii_xllcenter(filename_asc)
    
    # 2) Create an in-memory dataset.
    profile_mem = {
        "driver": "MEM",
        "height": height,
        "width": width,
        "count": 1,
        "dtype": str(data.dtype),
        "crs": src_crs,
        "transform": transform,
        "nodata": nodata_val
    }
    
    with MemoryFile() as memfile_src:
        with memfile_src.open(**profile_mem) as dataset_src:
            dataset_src.write(data, 1)
            
            # 3) Calculate the transformation and dimensions in the destination CRS.
            transform_dst, width_dst, height_dst = calculate_default_transform(
                src_crs, dst_crs, width, height, *dataset_src.bounds
            )
            
            # 4) Create the output profile for PNG (RGBA).
            profile_dst = dataset_src.profile.copy()
            profile_dst.update({
                "driver": "PNG",
                "crs": dst_crs,
                "transform": transform_dst,
                "width": width_dst,
                "height": height_dst,
                "count": 4,
                "dtype": "uint8"
            })
            
            # 5) Reproject the band.
            data_reproj = np.zeros((height_dst, width_dst), dtype=np.float32)
            with MemoryFile() as memfile_dst:
                with memfile_dst.open(**profile_dst) as dataset_dst:
                    reproject(
                        source=rasterio.band(dataset_src, 1),
                        destination=data_reproj,
                        src_transform=transform,
                        src_crs=src_crs,
                        dst_transform=transform_dst,
                        dst_crs=dst_crs,
                        resampling=Resampling.nearest
                    )
                    
                    # 6) Apply the colormap.
                    # Color gradient: light blue to dark blue for values from 0 to limit_val (0.9*global_max).
                    low_rgb = np.array([173, 216, 230], dtype=np.float32)  # Light blue
                    high_rgb = np.array([0, 0, 139], dtype=np.float32)      # Dark blue
                    limit_val = 0.9 * global_max
                    
                    band = data_reproj
                    valid_mask = band != nodata_val
                    if valid_mask.sum() == 0:
                        R = np.zeros_like(band, dtype=np.uint8)
                        G = np.zeros_like(band, dtype=np.uint8)
                        B = np.zeros_like(band, dtype=np.uint8)
                        A = np.zeros_like(band, dtype=np.uint8)
                    else:
                        min_val = 0.0
                        # Scale values linearly from 0 to limit_val.
                        scaled = ((band - min_val) / (limit_val - min_val)) * 255.0
                        scaled = np.clip(scaled, 0, 255).astype(np.float32)
                        above_mask = band > global_max  # Values above global_max become black.
                        R = low_rgb[0] + (high_rgb[0] - low_rgb[0]) * (scaled / 255.0)
                        G = low_rgb[1] + (high_rgb[1] - low_rgb[1]) * (scaled / 255.0)
                        B = low_rgb[2] + (high_rgb[2] - low_rgb[2]) * (scaled / 255.0)
                        R[above_mask] = 0
                        G[above_mask] = 0
                        B[above_mask] = 0
                        R = R.astype(np.uint8)
                        G = G.astype(np.uint8)
                        B = B.astype(np.uint8)
                        A = np.zeros_like(scaled, dtype=np.uint8)
                        A[valid_mask] = 255
                    dataset_dst.write(R, 1)
                    dataset_dst.write(G, 2)
                    dataset_dst.write(B, 3)
                    dataset_dst.write(A, 4)
                
                with open(output_png, 'wb') as f:
                    f.write(memfile_dst.read())
    
    # 7) Get the bounding box in EPSG:4326.
    left, bottom, right, top = dataset_src.bounds
    transformer = Transformer.from_crs(src_crs, "EPSG:4326", always_xy=True)
    lon_left, lat_bottom = transformer.transform(left, bottom)
    lon_right, lat_top = transformer.transform(right, top)
    
    return (lat_bottom, lon_left, lat_top, lon_right)

def format_filename(filename):
    """
    Extracts the number after "Depth____" in the filename and formats it as a 6-digit number.
    For example, returns "Depth_000123.png".
    """
    match = re.search(r"Depth____([\d\.]+)\.asc", filename)
    if match:
        num = float(match.group(1))
        num_int = int(num)
        formatted_num = f"{num_int:06d}"
        return f"Depth_{formatted_num}.png"
    return None

def extract_number(filename):
    """
    Extracts the number after "Depth____" from the filename and returns it as a float.
    Used to sort the files.
    """
    match = re.search(r"Depth____([\d\.]+)\.asc", filename)
    if match:
        return float(match.group(1))
    else:
        return float('inf')

# ====================================================
# MAIN PROCESSING
# ====================================================

# Create output directory if it does not exist
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

# List and sort ASC files in the input directory
asc_files = [f for f in os.listdir(INPUT_DIR) if f.endswith(".asc")]
asc_files = sorted(asc_files, key=extract_number)

bounding_boxes = []

# Convert each ASC file to PNG using the user-defined maximum depth
for filename in asc_files:
    input_path = os.path.join(INPUT_DIR, filename)
    output_name = format_filename(filename)
    if output_name is not None:
        output_path = os.path.join(OUTPUT_DIR, output_name)
        bbox_latlon = asc_to_png(
            filename_asc=input_path,
            output_png=output_path,
            src_crs=SRC_CRS,
            dst_crs=DST_CRS,
            global_max=USER_MAX_DEPTH
        )
        bounding_boxes.append((output_name, bbox_latlon))
        # print(f"{filename} => {output_name}, BBox WGS84 = {bbox_latlon}")

# --- Generate Vertical Legend as PNG ---
# Legend dimensions and margins (in pixels)
legend_width = LEGEND_WIDTH      # e.g., 50 px wide
legend_height = 256              # e.g., 256 px tall
top_margin = 40                  # Space for the title
bottom_margin = 20               # Space for the labels
left_margin = 25
right_margin = 10
inner_height = legend_height - top_margin - bottom_margin

# Create an RGBA image for the legend background:
# The background will be white with 50% transparency.
legend_array = np.ones((legend_height, legend_width, 4), dtype=np.uint8) * 255
legend_array[..., 3] = 128  # 128/255 = 50% opacity for the background

# Draw the color bar vertically (fully opaque):
low_rgb = np.array([173, 216, 230], dtype=np.float32)  # Light blue
high_rgb = np.array([0, 0, 139], dtype=np.float32)      # Dark blue
limit_val = 0.9 * USER_MAX_DEPTH

for y in range(inner_height):
    factor = y / float(inner_height - 1)
    inv_factor = 1 - factor  # So that the top (at y = 0) is dark blue
    color = low_rgb + (high_rgb - low_rgb) * inv_factor
    color = np.clip(color, 0, 255).astype(np.uint8)
    # Fill the color bar area with full opacity (alpha = 255)
    for x in range(left_margin, legend_width - right_margin):
        legend_array[top_margin + y, x, :3] = color
        legend_array[top_margin + y, x, 3] = 255  # Fully opaque for the color bar

# Convert the numpy array to a PIL image (RGBA)
legend_img = Image.fromarray(legend_array, mode="RGBA")
draw = ImageDraw.Draw(legend_img)
font = ImageFont.load_default()

# Draw title (centered, fully opaque text)
title_text = "Depth [m]"
bbox_title = draw.textbbox((0, 0), title_text, font=font)
w_title = bbox_title[2] - bbox_title[0]
h_title = bbox_title[3] - bbox_title[1]
draw.text(((legend_width - w_title) // 2, 2), title_text, font=font, fill=(0, 0, 0, 255))

# Draw lower label (0)
min_label = "0"
bbox_min = draw.textbbox((0, 0), min_label, font=font)
w_min = bbox_min[2] - bbox_min[0]
draw.text((left_margin - 10, legend_height - bottom_margin - h_title), min_label, font=font, fill=(0, 0, 0, 255))

# Draw upper label (limit value: 90% of USER_MAX_DEPTH)
max_label = f"{limit_val:.0f}"
bbox_max = draw.textbbox((0, 0), max_label, font=font)
w_max = bbox_max[2] - bbox_max[0]
draw.text((left_margin - 10, top_margin - h_title), max_label, font=font, fill=(0, 0, 0, 255))

legend_output = os.path.join(OUTPUT_DIR, "legend.png")
legend_img.save(legend_output)
print("Legend saved at", legend_output)

print("\nProcess completed!")
print("PNG files generated with names like 'Depth_******.png'.")

Legend saved at /Users/cmgotelli/Desktop/dems/legend.png

Process completed!
PNG files generated with names like 'Depth_******.png'.


## Create HTML

In [9]:
# ========================================
# Example snippet for build_map.ipynb
# Updated to include a second animation layer (with the same animation)
# and allow customization of the HTML title and layer names.
# ========================================

output_html_filename = "my_map.html"

# User-defined variables for customization:
html_title = "My Custom Map Title"       # Title of the HTML page
animation_layer_name = "T = 10 years"  # Name for the first animation layer
animation_layer2_name = "T = 100 years" # Name for the second animation layer
drawings_layer_name = "Drawings Layer"      # Name for the drawings layer

# Define the GitHub folder URL (the one seen in the browser)
github_url = "https://github.com/cgotelli/Floods-WebMap/tree/main/simulation_files/png"

# Legend image location in the new repo:
legend_url = "https://github.com/cgotelli/Floods-WebMap/raw/refs/heads/main/legend.png"

default_zoom = 16
overlay_opacity = 0.75
start_year = 2025
start_month = 1
start_day = 1

# Convert the GitHub URL to the corresponding API URL.
# GitHub URL format:
#   https://github.com/{user}/{repo}/tree/{branch}/{path}
# API URL format:
#   https://api.github.com/repos/{user}/{repo}/contents/{path}
if "github.com" in github_url:
    # Remove the "https://github.com/" prefix.
    path_part = github_url.replace("https://github.com/", "")
    # Split the remaining part.
    parts = path_part.split("/")
    # Expecting parts to be: [user, repo, "tree", branch, path...]
    if len(parts) >= 5 and parts[2] == "tree":
        user = parts[0]
        repo = parts[1]
        branch = parts[3]  # branch is not used in API URL
        folder_path = "/".join(parts[4:])
        folder_url = f"https://api.github.com/repos/{user}/{repo}/contents/{folder_path}"
    else:
        # Fallback to the provided URL if the format is unexpected
        folder_url = github_url
else:
    folder_url = github_url

# Suppose we've computed bounding_boxes from the ASC->PNG script.
# We'll use the first bounding box for demonstration:
if 'bounding_boxes' in globals() and bounding_boxes:
    # bounding_boxes[i] => (output_png_name, (lat_bottom, lon_left, lat_top, lon_right))
    _, (lat_bottom, lon_left, lat_top, lon_right) = bounding_boxes[0]
else:
    # Fallback or default values
    lat_bottom, lon_left, lat_top, lon_right = 46.5199, 6.6309, 46.5211, 6.6363

# Build the JavaScript strings for the Leaflet bounds and center.
imageBounds_str = f"[[{lat_bottom}, {lon_left}], [{lat_top}, {lon_right}]]"
center_lat = 0.5 * (lat_bottom + lat_top)
center_lon = 0.5 * (lon_left + lon_right)
mapCenter_str = f"[{center_lat:.5f}, {center_lon:.5f}]"

# Start date for frames (JS-compatible)
startDateJS = f"new Date('{start_year:04d}-{start_month:02d}-{start_day:02d}T00:00:00Z')"

# Build the HTML content with all text in English and customized title and layer names
html_content = f"""<!DOCTYPE html>
<html>
<head>
  <title>{html_title}</title>
  <meta charset="utf-8"/>
  <!-- CSS: Leaflet -->
  <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/leaflet@1.5.1/dist/leaflet.css" />
  <!-- CSS: Leaflet TimeDimension -->
  <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/leaflet-timedimension@1.1.1/dist/leaflet.timedimension.control.min.css" />
  <!-- CSS: Leaflet Measure -->
  <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/leaflet-measure@3.1.0/dist/leaflet-measure.min.css">
  <!-- CSS: Leaflet.draw (v0.4.2) -->
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/leaflet.draw/0.4.2/leaflet.draw.css"/>
  <!-- CSS: FontAwesome for icons -->
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/4.7.0/css/font-awesome.min.css">
  <style>
    #map {{
      position: relative;
      width: 100%;
      height: 600px;
    }}
  </style>
</head>
<body>
  <div id="map">
    <!-- Legend in the lower-right corner -->
    <div id="legend" style="position: absolute; bottom: 15px; right: 10px; background: rgba(255,255,255,0); padding: 5px; border: 0px solid #ccc; z-index: 1000;">
      <img src="{legend_url}" alt="Depth Legend">
    </div>
  </div>
  
  <!-- JS: Leaflet, Iso8601, TimeDimension, Measure, Leaflet.draw -->
  <script src="https://cdn.jsdelivr.net/npm/leaflet@1.5.1/dist/leaflet.js"></script>
  <script src="https://cdn.jsdelivr.net/npm/iso8601-js-period@0.2.1/iso8601.min.js"></script>
  <script src="https://cdn.jsdelivr.net/npm/leaflet-timedimension@1.1.1/dist/leaflet.timedimension.min.js"></script>
  <script src="https://cdn.jsdelivr.net/npm/leaflet-measure@3.1.0/dist/leaflet-measure.min.js"></script>
  <script src="https://cdnjs.cloudflare.com/ajax/libs/leaflet.draw/0.4.2/leaflet.draw.js"></script>
  
  <script>
    /*************************************************
     * preloadImages: loads all image URLs in parallel
     *************************************************/
    function preloadImages(urls, callback) {{
      let loadedCount = 0;
      const total = urls.length;
      urls.forEach(url => {{
        const img = new Image();
        img.onload = img.onerror = function() {{
          loadedCount++;
          if (loadedCount === total) {{
            callback(urls);
          }}
        }};
        img.src = url;
      }});
    }}
    
    /*************************************************
     * getPNGFilesFromGitHubFolder: fetch Depth_*.png from new repo
     *************************************************/
    function getPNGFilesFromGitHubFolder(folderUrl, callback) {{
      fetch(folderUrl)
        .then(response => response.json())
        .then(json => {{
          let pngFiles = json.filter(item => 
            item.type === "file" && item.name.toLowerCase().startsWith("depth")
          ).map(item => item.download_url);
          pngFiles.sort();
          callback(pngFiles);
        }})
        .catch(error => {{
          console.error("Error fetching GitHub folder:", error);
          callback([]);
        }});
    }}
    
    /*************************************************
     * Subclass for ImageOverlay in TimeDimension
     *************************************************/
    L.TimeDimension.Layer.ImageOverlay = L.TimeDimension.Layer.extend({{
      initialize: function(layer, options) {{
        L.TimeDimension.Layer.prototype.initialize.call(this, layer, options);
        if (typeof this.options.time === 'string') {{
          this._time = new Date(this.options.time).getTime();
        }} else if (this.options.time instanceof Date) {{
          this._time = this.options.time.getTime();
        }} else {{
          this._time = this.options.time;
        }}
      }},
      _onNewTimeLoading: function(ev) {{
        this.fire('timeload', {{ time: ev.time }});
      }},
      isReady: function(time) {{
        return true;
      }},
      _update: function() {{
        if (!this._map) return;
        var currentTime = this._timeDimension.getCurrentTime();
        var tolerance = 500;
        console.log("Overlay time: " + this._time + ", currentTime: " + currentTime);
        if (Math.abs(currentTime - this._time) < tolerance) {{
          if (!this._map.hasLayer(this._baseLayer)) {{
            console.log("Adding overlay for time " + this._time);
            this._map.addLayer(this._baseLayer);
          }}
        }} else {{
          if (this._map.hasLayer(this._baseLayer)) {{
            console.log("Removing overlay for time " + this._time);
            this._map.removeLayer(this._baseLayer);
          }}
        }}
        return true;
      }}
    }});
    L.timeDimension.layer.imageOverlay = function(layer, options) {{
      return new L.TimeDimension.Layer.ImageOverlay(layer, options);
    }};
    
    /*************************************************
     * Initialization
     *************************************************/
    var folderUrl = "{folder_url}";
    
    getPNGFilesFromGitHubFolder(folderUrl, function(pngFiles) {{
      console.log("Obtained PNG URLs:", pngFiles);
      
      preloadImages(pngFiles, function(loadedUrls) {{
        console.log("All images have been preloaded.");
        
        var times = [];
        var startTime = {startDateJS};
        for (var i = 0; i < loadedUrls.length; i++) {{
          var time = new Date(startTime.getTime() + i * 1000);
          times.push(time.toISOString());
        }}
        
        var imageBounds = {imageBounds_str};
        var mapCenter = {mapCenter_str};
        
        var mapStartTime = times[0];
        var mapEndTime = times[times.length - 1];
        
        var map = L.map('map', {{
          center: mapCenter,
          zoom: {default_zoom},
          timeDimension: true,
          timeDimensionOptions: {{
            timeInterval: mapStartTime + "/" + mapEndTime,
            period: "PT1S",
            currentTime: new Date(mapStartTime).getTime()
          }}
        }});
        
        L.control.timeDimension({{
          autoPlay: false,
          loopButton: true,
          timeSliderDragUpdate: true,
          speedSlider: true,
          playerOptions: {{
            transitionTime: 100,
            loop: true,
            startOver: true
          }},
          minSpeed: 0.1,
          maxSpeed: 20,
          displayDate: false
        }}).addTo(map);
        
        /********************
         * Base layers
         ********************/
        var osmStandard = L.tileLayer('https://{{s}}.tile.openstreetmap.org/{{z}}/{{x}}/{{y}}.png', {{ 
          attribution: '© OpenStreetMap Contributors' 
        }});
        var swissColor = L.tileLayer(
          'https://wmts.geo.admin.ch/1.0.0/ch.swisstopo.pixelkarte-farbe/default/current/3857/{{z}}/{{x}}/{{y}}.jpeg',
          {{ 
            attribution: '© swisstopo <img src="https://upload.wikimedia.org/wikipedia/commons/f/f3/Flag_of_Switzerland.svg" style="height:16px; vertical-align:middle;" alt="Swiss Flag">' 
          }}
        );
        var swissTopo = L.tileLayer(
          'https://wmts.geo.admin.ch/1.0.0/ch.swisstopo.swissimage/default/current/3857/{{z}}/{{x}}/{{y}}.jpeg',
          {{ 
            attribution: '© swisstopo <img src="https://upload.wikimedia.org/wikipedia/commons/f/f3/Flag_of_Switzerland.svg" style="height:16px; vertical-align:middle;" alt="Swiss Flag">' 
          }}
        );
        var cartoPositron = L.tileLayer(
          'https://{{s}}.basemaps.cartocdn.com/light_all/{{z}}/{{x}}/{{y}}{{r}}.png',
          {{ attribution: '© OpenStreetMap contributors, © CartoDB' }}
        );
        var cartoDark = L.tileLayer(
          'https://{{s}}.basemaps.cartocdn.com/dark_all/{{z}}/{{x}}/{{y}}{{r}}.png',
          {{ attribution: '© OpenStreetMap contributors, © CartoDB' }}
        );
        
        var baseLayers = {{
          "OSM Standard": osmStandard,
          "Swiss Color": swissColor,
          "Swiss Topo": swissTopo,
          "CartoDB Positron": cartoPositron,
          "CartoDB Dark Matter": cartoDark
        }};
        
        /********************
         * Overlay layers
         ********************/
        // FeatureGroup for drawings (Leaflet.draw)
        var drawnItems = new L.FeatureGroup();
        map.addLayer(drawnItems);
        
        // Animation layer 1: group all time-based image overlays
        var animationLayer = L.layerGroup();
        for (let i = 0; i < loadedUrls.length; i++) {{
          var overlay = L.imageOverlay(loadedUrls[i], imageBounds, {{ opacity: {overlay_opacity} }});
          var tdLayer = L.timeDimension.layer.imageOverlay(overlay, {{ time: times[i] }});
          animationLayer.addLayer(tdLayer);
        }}
        // Add the first animation layer to the map by default
        animationLayer.addTo(map);
        
        // Animation layer 2: identical to the first (for demonstration)
        var animationLayer2 = L.layerGroup();
        for (let i = 0; i < loadedUrls.length; i++) {{
          var overlay = L.imageOverlay(loadedUrls[i], imageBounds, {{ opacity: {overlay_opacity} }});
          var tdLayer = L.timeDimension.layer.imageOverlay(overlay, {{ time: times[i] }});
          animationLayer2.addLayer(tdLayer);
        }}
        // Add the second animation layer to the map by default
        animationLayer2.addTo(map);
        
        // Layers control: base layers and overlays (with user-defined names)
        L.control.layers(baseLayers, {{
          "{animation_layer_name}": animationLayer,
          "{animation_layer2_name}": animationLayer2,
          "{drawings_layer_name}": drawnItems
        }}).addTo(map);
        
        /********************
         * Measure control
         ********************/
        var measureControl = new L.Control.Measure({{
          primaryLengthUnit: 'meters',
          primaryAreaUnit: 'hectares',
          position: 'topleft'
        }});
        measureControl.addTo(map);
        
        // Add one of the base layers by default
        swissTopo.addTo(map);
        
        /*************************************************
         * Leaflet.draw
         *************************************************/
        var drawControl = new L.Control.Draw({{
          edit: {{ featureGroup: drawnItems }},
          draw: {{
            polygon: true,
            polyline: true,
            rectangle: true,
            circle: true,
            marker: true
          }}
        }});
        map.addControl(drawControl);
        
        // --- New Control: Button to save drawings ---
        var saveDrawingsControl = L.Control.extend({{
          options: {{
            position: 'topleft'
          }},
          onAdd: function(map) {{
            var container = L.DomUtil.create('div', 'leaflet-bar leaflet-control');
            container.style.backgroundColor = 'white';
            container.style.padding = '10px';
            container.style.cursor = 'pointer';
            container.title = 'Save drawings';
            container.innerHTML = '<i class="fa fa-save"></i>';
            L.DomEvent.disableClickPropagation(container);
            L.DomEvent.on(container, 'click', function(e) {{
              // Convert the drawings layer to GeoJSON
              var data = drawnItems.toGeoJSON();
              var convertedData = JSON.stringify(data, null, 2);
              // Create a Blob and generate a URL for download
              var blob = new Blob([convertedData], {{ type: 'application/json' }});
              var url = URL.createObjectURL(blob);
              var a = document.createElement('a');
              a.href = url;
              a.download = 'drawings.geojson';
              document.body.appendChild(a);
              a.click();
              document.body.removeChild(a);
              URL.revokeObjectURL(url);
            }});
            return container;
          }}
        }});
        map.addControl(new saveDrawingsControl());
        // --- End of save control ---
        
        map.on('draw:created', function(e) {{
          var type = e.layerType,
              layer = e.layer;
          drawnItems.addLayer(layer);
          var infoText = "";
          if (type === 'polygon' || type === 'rectangle') {{
            var latlngs = layer.getLatLngs();
            if (Array.isArray(latlngs[0])) {{ latlngs = latlngs[0]; }}
            var area = L.GeometryUtil.geodesicArea(latlngs);
            var perimeter = 0;
            for (var j = 0; j < latlngs.length; j++) {{
              var next = latlngs[(j + 1) % latlngs.length];
              perimeter += latlngs[j].distanceTo(next);
            }}
            infoText = "Area: " + area.toFixed(2) + " m², Perimeter: " + perimeter.toFixed(2) + " m";
          }} else if (type === 'circle') {{
            var r = layer.getRadius();
            var area = Math.PI * r * r;
            var circumference = 2 * Math.PI * r;
            infoText = "Circle Area: " + area.toFixed(2) + " m², Circumference: " + circumference.toFixed(2) + " m";
          }} else if (type === 'polyline') {{
            var latlngs = layer.getLatLngs();
            var length = 0;
            for (var k = 0; k < latlngs.length - 1; k++) {{
              length += latlngs[k].distanceTo(latlngs[k+1]);
            }}
            infoText = "Length: " + length.toFixed(2) + " m";
          }} else if (type === 'marker') {{
            var latlng = layer.getLatLng();
            infoText = "Coordinates: " + latlng.lat.toFixed(5) + ", " + latlng.lng.toFixed(5);
          }}
          layer.bindPopup(infoText).openPopup();
        }});
      }});
    }});
  </script>
</body>
</html>
"""

# Finally, write the html_content to a file
with open(output_html_filename, "w", encoding="utf-8") as f:
    f.write(html_content)

print(f"HTML updated and exported to '{output_html_filename}'.")

HTML updated and exported to 'my_map.html'.
